In [1]:
import numpy as np
from qiskit.circuit import ParameterVector
from squlearn.expectation_operator import SummedPaulis
from qiskit.quantum_info import SparsePauliOp

from squlearn.util.optree.optree_derivative import operator_derivative

In [2]:

op = SummedPaulis(5)
x=ParameterVector('x', op.num_parameters)
e = op.get_pauli(x)
print(e)

SummedOp([
  1.0*x[0] * IIIII,
  1.0*x[1] * IIIIZ,
  1.0*x[2] * IIIZI,
  1.0*x[3] * IIZII,
  1.0*x[4] * IZIII,
  1.0*x[5] * ZIIII
])


In [3]:
e_sparse = op.get_pauli_new(x)
print(e_sparse)

SparsePauliOp(['IIIII', 'IIIIZ', 'IIIZI', 'IIZII', 'IZIII', 'ZIIII'],
              coeffs=[ParameterExpression(1.0*x[0]), ParameterExpression(1.0*x[1]),
 ParameterExpression(1.0*x[2]), ParameterExpression(1.0*x[3]),
 ParameterExpression(1.0*x[4]), ParameterExpression(1.0*x[5])])


In [4]:
e_sparse.coeffs


array([ParameterExpression(1.0*x[0]), ParameterExpression(1.0*x[1]),
       ParameterExpression(1.0*x[2]), ParameterExpression(1.0*x[3]),
       ParameterExpression(1.0*x[4]), ParameterExpression(1.0*x[5])],
      dtype=object)

In [5]:
test = e_sparse.coeffs[0]._symbol_expr
print("debug")

debug


In [6]:
e_sparse.paulis

PauliList(['IIIII', 'IIIIZ', 'IIIZI', 'IIZII', 'IZIII', 'ZIIII'])

In [7]:
operator_derivative(e_sparse,x[2])

d_coeff 0.0 <class 'float'> IIIII
d_coeff 0.0 <class 'float'> IIIIZ
d_coeff 1.0 <class 'float'> IIIZI
d_coeff 0.0 <class 'float'> IIZII
d_coeff 0.0 <class 'float'> IZIII
d_coeff 0.0 <class 'float'> ZIIII


SparsePauliOp(['IIIZI'],
              coeffs=[1.+0.j])

In [8]:
op = SparsePauliOp(["II","ZZ"],[(0.5+1j)*x[0]*x[2],np.arccos(x[0])*x[2]])
operator_derivative(op,x[0])


d_coeff x[2]*(0.5 + 1.0*I) <class 'qiskit.circuit.parameterexpression.ParameterExpression'> II
d_coeff 1.0*x[2] <class 'qiskit.circuit.parameterexpression.ParameterExpression'> ZZ


SparsePauliOp(['II', 'ZZ'],
              coeffs=[ParameterExpression(x[2]*(0.5 + 1.0*I)), ParameterExpression(1.0*x[2])])

In [9]:
SparsePauliOp([],[])

QiskitError: 'Input Pauli list is empty.'

In [ ]:
test = np.array([1.0*x[0],1.0*x[1]])
test

In [ ]:
for i,c in enumerate(test):
    print(i,c,c._symbol_expr)
    print(c.gradient(x[1]))

In [ ]:
test = e_sparse.coeffs
test

In [ ]:
for i,c in enumerate(test):
    print(i,c)
    print(c.gradient(x[1]))